**This notebook loops through a Fabric Lakehouse table containing McDonalds reviews. It sends the reviews to Azure OpenAI using the native OpenAI endpoint in Fabric and has it categorize the reviews based on a list of categories provided while also providing sentiment**

In [ ]:
#Install the openai library we need for this notebook
%pip install openai==0.28.1

In [2]:
#Specify the categories to use seperated by a comma
categories = "'Order Delivery Speed','Order Accuracy', 'Food Quality', 'Neighborhood or Location', 'Cleanliness', 'Cost', 'Other'"

StatementMeta(, 251d4d35-d46d-42dc-98fb-615c5136b48b, 10, Finished, Available)

In [ ]:
#Read the reviews from the Fabric lakehouse table
df = spark.sql("SELECT * FROM Reviews_Lakehouse.mcdonalds_reviews")
display(df)

In [ ]:
import pandas as pd
import openai
import json
from delta.tables import *
from pyspark.sql.functions import *
from delta.tables import *

#Pick only the relevant fields
reviews_df = df.select("review","_unit_id" )
df_pandas = reviews_df.toPandas()

#Create some new columns in the dataframe to store the results
df_pandas['review_topics'] = ' '
df_pandas['review_sentiment'] = ' '


#Loop through each row 
for index, row in df_pandas.iterrows():

    #Get the review text from the current row
    review = row['review']
    

    #Build the prompt for Azure OpenAI
    prompt = f"""
      You are a restaurant review analysis bot. Given the following categories: {categories}.
      Respond to the user review data with a category or categories from the list that fits the review comments.
      Only respond with the provided category names and whether the overall sentiment for the review is positive, negative, or neutral.
      Put the response in JSON format as shown here:
    . Example: 'It took 20 minutes to get my order! That is unacceptable. The food also didn't taste very good'
      Response: {{"response": {{"categories":"Order Delivery Speed, Food Quality", "sentiment": "Negative"}} "
    """

    response = openai.ChatCompletion.create(
        deployment_id='gpt-35-turbo', # deployment_id could be one of {gpt-35-turbo, gpt-35-turbo-16k}
        messages = [{
	    	"role": "system",
		    "content": prompt
	    },
	    {
	    	"role": "user",
	    	"content": review
    }],
    temperature=0.0,
    max_tokens=2000,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stream=False,
    stop=None)


  

    #Get the response back from Azure OpenAI and store it in a json variable
    response = response['choices'][0]['message']['content']
    data = json.loads(response)

    # Access the responses in the JSON
    for item in data:
        categories = data['response']['categories']
        sentiment = data['response']['sentiment']

    # Add the responses to the pandas dataframe
    df_pandas.loc[index, 'review_topics'] = categories
    df_pandas.loc[index, 'review_sentiment'] = sentiment


# Change back to a Spark dataframe
df_updates = spark.createDataFrame(df_pandas)

# Load the data currently in the table
deltaTableReviews = DeltaTable.forPath(spark, 'abfss://9ae66882-2572-4456-9b4e-cfdfcb0db0c3@onelake.dfs.fabric.microsoft.com/098b9932-65df-430d-b79c-0ad0181f2162/Tables/mcdonalds_reviews')

# This does a Delta merge to update each row with the sentiment and topics / categories 
deltaTableReviews.alias('reviews_table') \
    .merge(
        df_updates.alias('updates'),
        'reviews_table._unit_id = updates._unit_id'
    ) \
    .whenMatchedUpdate(set = {'reviews_table.review_topics' : 'updates.review_topics', 'reviews_table.review_sentiment' : 'updates.review_sentiment'}) \
    .execute()



In [ ]:
#Read the reviews from the Fabric lakehouse table
df2 = spark.sql("SELECT * FROM Reviews_Lakehouse.mcdonalds_reviews LIMIT 10")
display(df2)

In [ ]:
# If you want to write the results to another table, you can do that using the code below. Just provide a table name.

delta_table_name = "review_category_and_sentiment"
df_updates.write.mode("overwrite").format("delta").saveAsTable(delta_table_name)